<a href="https://colab.research.google.com/github/pioneer123456/Smart-ML-based-Refrigerator/blob/main/Training_For_Raspberry_pi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Models for edge device(Raspberry-pi)

Install the required packages

In [1]:
!pip install -q tflite-model-maker
!pip install -q tflite-support

     |████████████████████████████████| 642 kB 7.6 MB/s 
     |████████████████████████████████| 87 kB 6.0 MB/s 
     |████████████████████████████████| 1.1 MB 62.2 MB/s 
     |████████████████████████████████| 6.4 MB 57.6 MB/s 
     |████████████████████████████████| 596 kB 44.7 MB/s 
     |████████████████████████████████| 1.2 MB 44.2 MB/s 
     |████████████████████████████████| 237 kB 57.1 MB/s 
     |████████████████████████████████| 10.9 MB 36.5 MB/s 
     |████████████████████████████████| 120 kB 59.9 MB/s 
     |████████████████████████████████| 3.4 MB 54.9 MB/s 
     |████████████████████████████████| 77 kB 5.7 MB/s 
     |████████████████████████████████| 42.5 MB 91 kB/s 
     |████████████████████████████████| 840 kB 38.1 MB/s 
     |████████████████████████████████| 25.3 MB 1.4 MB/s 
     |████████████████████████████████| 47.8 MB 53 kB/s 
     |████████████████████████████████| 99 kB 9.9 MB/s 
     |████████████████████████████████| 352 kB 53.8 MB/s 
     |████████████████

In [2]:
!sudo apt-get install libportaudio2
!pip install sounddevice

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 42 not upgraded.
Need to get 64.6 kB of archives.
After this operation, 215 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Fetched 64.6 kB in 0s (234 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype


Import the required packages.

In [3]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

# Prepare the dataset
This dataset contains about 80 images of 2 type of banana figurines: a good condition and defective condition. 

We start with downloading the dataset.

In [4]:
!git clone https://github.com/pioneer123456/Smart-ML-based-Refrigerator.git

Cloning into 'Smart-ML-based-Refrigerator'...
remote: Enumerating objects: 259, done.
remote: Counting objects: 100% (259/259), done.
remote: Compressing objects: 100% (152/152), done.
remote: Total 259 (delta 125), reused 228 (delta 105), pack-reused 0
Receiving objects: 100% (259/259), 18.77 MiB | 20.21 MiB/s, done.
Resolving deltas: 100% (125/125), done.


In [5]:
!cp -r Smart-ML-based-Refrigerator/Data /content/fruit_data/

# Train the object detection model
## Step 1: Load the dataset
Images in train_data is used to train the custom object detection model.
Images in val_data is used to check if the model can generalize well to new images that it hasn't seen before.

In [6]:
train_data = object_detector.DataLoader.from_pascal_voc(
    '/content/fruit_data/Train_data',
    '/content/fruit_data/Train_data',
    ['looks_good', 'looks_defective']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    '/content/fruit_data/Test_data',
    '/content/fruit_data/Test_data',
    ['looks_good', 'looks_defective']
)

# Step 2: Select a model architecture
EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the EfficientDet architecture.

In [7]:
spec = model_spec.get('efficientdet_lite0')

# Step 3: Train the TensorFlow model with the training data.

In [8]:
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, epochs=20, validation_data=val_data)

Epoch 1/20
20/20 [==============================] - 76s 2s/step - det_loss: 1.6746 - cls_loss: 1.1012 - box_loss: 0.0115 - reg_l2_loss: 0.0630 - loss: 1.7376 - learning_rate: 0.0065 - gradient_norm: 2.2768 - val_det_loss: 1.5659 - val_cls_loss: 1.0439 - val_box_loss: 0.0104 - val_reg_l2_loss: 0.0630 - val_loss: 1.6289
Epoch 2/20
20/20 [==============================] - 40s 2s/step - det_loss: 1.4232 - cls_loss: 0.9779 - box_loss: 0.0089 - reg_l2_loss: 0.0630 - loss: 1.4862 - learning_rate: 0.0049 - gradient_norm: 4.0561 - val_det_loss: 1.3871 - val_cls_loss: 0.8309 - val_box_loss: 0.0111 - val_reg_l2_loss: 0.0630 - val_loss: 1.4501
Epoch 3/20
20/20 [==============================] - 40s 2s/step - det_loss: 1.1486 - cls_loss: 0.7697 - box_loss: 0.0076 - reg_l2_loss: 0.0630 - loss: 1.2117 - learning_rate: 0.0048 - gradient_norm: 3.3984 - val_det_loss: 1.8140 - val_cls_loss: 1.2694 - val_box_loss: 0.0109 - val_reg_l2_loss: 0.0631 - val_loss: 1.8770
Epoch 4/20
20/20 [======================

# Step 4. Evaluate the model with the validation data.

In [9]:
model.evaluate(val_data)

1/1 [==============================] - 7s 7s/step



{'AP': 0.23814964,
 'AP50': 0.57475245,
 'AP75': 0.25219777,
 'AP_/looks_defective': 0.18592384,
 'AP_/looks_good': 0.29037544,
 'APl': 0.23831426,
 'APm': -1.0,
 'APs': -1.0,
 'ARl': 0.39141414,
 'ARm': -1.0,
 'ARmax1': 0.34949496,
 'ARmax10': 0.3641414,
 'ARmax100': 0.39141414,
 'ARs': -1.0}


# Step 5: Export as a TensorFlow Lite model.

In [10]:
model.export(export_dir='.', tflite_filename='android1.tflite')

# Step 6: Evaluate the TensorFlow Lite model.

In [11]:
model.evaluate_tflite('android1.tflite', val_data)

20/20 [==============================] - 57s 3s/step



{'AP': 0.24299856,
 'AP50': 0.58851624,
 'AP75': 0.26895052,
 'AP_/looks_defective': 0.19169362,
 'AP_/looks_good': 0.2943035,
 'APl': 0.24314547,
 'APm': -1.0,
 'APs': -1.0,
 'ARl': 0.42070708,
 'ARm': -1.0,
 'ARmax1': 0.32575756,
 'ARmax10': 0.42070708,
 'ARmax100': 0.42070708,
 'ARs': -1.0}

In [12]:
# Download the TFLite model to your local computer.
from google.colab import files
files.download('android1.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>